In [5]:
suppressPackageStartupMessages(library(cicero))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(parallel))

In [6]:
clus <- makeCluster(32)
#clusterExport(clus)

In [7]:
wd = '/nfs/lab/projects/pbmc_snATAC/pipeline/snATAC/combined_files/pbmc1to6'
setwd(wd)

In [8]:
sc.umap <- read.table( 'pbmc.merged.cluster_labels.txt', sep='\t', header=T, row.names=1) ### map barcode to cluster 

In [9]:
clusters <- unique(sc.umap$cluster)

In [10]:
clusters

[1] NK-cell       Monocyte      CD8-T-cell    CD4-T-cell    B-cell       
[6] pDC           Megakaryocyte
Levels: B-cell CD4-T-cell CD8-T-cell Megakaryocyte Monocyte NK-cell pDC

In [9]:
zz   = gzfile(paste0('cluster_peaks/', cluster, '.merged_peaks.long_fmt.mtx.gz'),'rt')   ### long for matrix peak x barcode
data = read.table(zz,header=T,  sep="\t")

In [14]:
### convert to sparse matrix peak x barcode
sc.data <- with(data, sparseMatrix(i=as.numeric(peak), j=as.numeric(barcode), 
                                   x=value, dimnames=list(levels(peak), levels(barcode))))
rownames(sc.data) <- paste0('chr', gsub('-','_', gsub(':','_',rownames(sc.data))))


In [59]:
# cluster_peaks = read.table("pbmc1.sorted.merged.bed") ### peaks called in each cluster to convert as wide format

# mat   = str_split_fixed( cluster_peaks[,4], "\\,", length(unique(sc.umap$cluster)))
# mat2  = t( apply( mat, 1, function(x) as.numeric(unique(sc.umap$cluster) %in% x)))

# peaks = cbind(paste(cluster_peaks[,1], cluster_peaks[,2],  cluster_peaks[,3], sep="_"), mat2)
# colnames(peaks) = c("peak", unique(as.character(sc.umap$cluster)))

In [17]:
for (cluster in clusters) {
    prefix = cluster
    sc.data.subset <- sc.data
    cellinfo <-data.frame(cells=colnames(sc.data.subset))
    row.names(cellinfo) <- cellinfo$cells
    dhsinfo <- data.frame(site_name=rownames(sc.data.subset))
    row.names(dhsinfo) <- dhsinfo$site_name
    dhsinfo <- cbind(dhsinfo, stringr::str_split_fixed(dhsinfo$site_name, "_", 3))
    names(dhsinfo) <- c('site_name','chr','bp1','bp2')
    dhsinfo$chr <- gsub('chr','', dhsinfo$chr)
    dhsinfo$bp1 <- as.numeric(as.character(dhsinfo$bp1))
    dhsinfo$bp2 <- as.numeric(as.character(dhsinfo$bp2))
    
    input_cds <- suppressWarnings(newCellDataSet(as(sc.data.subset, 'dgCMatrix'),
                                phenoData = methods::new('AnnotatedDataFrame', data = cellinfo),
                                featureData = methods::new('AnnotatedDataFrame', data = dhsinfo),
                                expressionFamily=negbinomial.size(),
                                lowerDetectionLimit=0))
    input_cds@expressionFamily <- binomialff()
    input_cds@expressionFamily@vfamily <- 'binomialff'
    input_cds <- detectGenes(input_cds)
    input_cds <- estimateSizeFactors(input_cds)
    
    input_cds <- input_cds[fData(input_cds)$num_cells_expressed > 0,]
    umap_coords <- sc.umap[colnames(sc.data.subset), c('UMAP1','UMAP2')]
    colnames(umap_coords) <- NULL
    
    cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = umap_coords, k=30)
    window <- 1e6
    data('human.hg19.genome')
    distance_parameters <- estimate_distance_parameter(cicero_cds, window=window, maxit=100, sample_num=100, distance_constraint=500000, genomic_coords=human.hg19.genome)
    mean_distance_parameter <- mean(unlist(distance_parameters))
    cicero_out <- generate_cicero_models(cicero_cds, distance_parameter=mean_distance_parameter, window=window, genomic_coords=human.hg19.genome)
    conns <- assemble_connections(cicero_out, silent=FALSE)
    #saveRDS(conns, file.path(wd, paste0('cicero/', cluster, '.1MB_cicero_conns.rds')))
    #write.table(conns, file.path(wd, paste0('cicero/', cluster, '.cicero_conns.txt')), sep='\t', quote=FALSE, row.names=FALSE)

## this step is to remove duplicated connections
conns = conns[order(-conns$coaccess),]
bed = cbind(str_split_fixed(conns[,1], "\\_", 3 ), str_split_fixed(conns[,2], "\\_", 3 ))
    #### mistake here because the order was not numeric ### must fix it
    
ord = matrix(parRapply(clus, bed, function(x) x[order(x)] ), ncol=6, byrow=T)
ord = cbind(ord[, c(5,1:2,5,3:4)], conns$coaccess)
dedup = ord[!duplicated(ord),]
dedup = data.frame( Peak1 = paste(dedup[,1], dedup[,2], dedup[,3], sep="_")  , 
                    Peak2 = paste(dedup[,4], dedup[,5], dedup[,6], sep="_") , coaccess = dedup[,7]  )
              
              
  write.table(dedup, file.path(wd, paste0('cicero/', cluster, '.cicero_conns_dedup.txt')), sep='\t', quote=FALSE, row.names=FALSE)
             
              
}

In [13]:
for (cluster in clusters){
dep = fread(file.path(wd, paste0('cicero/', cluster, '.cicero_conns_dedup.txt')), header=T, sep="\t", data.table=F)
dep = dep[complete.cases(dep),]


    
bed = cbind(str_split_fixed(dep[,1], "\\_", 3 ), str_split_fixed(dep[,2], "\\_", 3 ))

### reorder the peaks (before they were character)
ord = matrix(parRapply(clus, bed, function(x) x[order(as.numeric(x))] ), ncol=6, byrow=T)


conns = data.frame(Peak1 = paste(ord[,5], ord[,1], ord[,2], sep="_"), 
                 Peak2 = paste(ord[,5], ord[,3], ord[,4], sep="_"), coaccess = dep[,3])
                       
                       
write.table(conns, file.path(wd, paste0('cicero/', cluster, '.cicero_conns_dedup.txt')), sep='\t', quote=FALSE, row.names=FALSE)
  }
                       

In [12]:
head(dep)

Peak1,Peak2,coaccess
<fct>,<fct>,<dbl>
chr13_25479404_25479712,chr13_25480171_25480442,0.9987822
chr7_47852055_47852302,chr7_47852648_47852848,0.9981479
chr12_11863802_11864002,chr12_11864453_11865152,0.9972317
chr9_114767699_114767909,chr9_114768325_114768549,0.9969485
chr2_196195101_196195301,chr2_196195322_196195646,0.9962616
chr10_129670769_129671113,chr10_129671171_129671485,0.9953127


### General co-acessibility statistics

In [242]:
clusters <- unique(sc.umap$cluster)

In [308]:
sumatab = matrix(0, ncol=2, nrow= length(clusters))
rownames(sumatab) = clusters
for (cluster in clusters){
dep = read.table(file.path(wd, paste0('cicero/', cluster, '.cicero_conns_dedup.txt')), header=T, sep="\t")
dep = dep[complete.cases(dep),]
coa = dep[dep$coaccess >0.05, ]
sumatab[cluster,1] = nrow(dep)
sumatab[cluster,2] = nrow(coa)

    
# bed = cbind(str_split_fixed(coa[,1], "\\_", 3 ), str_split_fixed(coa[,2], "\\_", 3 ))

# ### reorder the peaks (before they were character)
# ord = matrix(parRapply(clus, bed, function(x) x[order(as.numeric(x))] ), ncol=6, byrow=T)
# ord = ord[, c(5,1:2,5,3:4)]

# dist = data.frame(span = as.numeric(ord[,6]) - as.numeric(ord[,2]), score = coa$coaccess)
# #dist$size_bin = cut(dist$span, breaks = 5 )
# #boxplot(score~size_bin, dist, las=2)
# #nam = parRapply(clus, ord,  function(x) paste(x, collapse="_"))

# long = data.frame(rbind(ord[,1:3], ord[,4:6]))

# long$conn = paste( rep(1:nrow(ord),2),  rep(c('a','b'), each = nrow(ord)), sep=".")
# write.table(long, "bed_tempor", sep="\t", quote=FALSE, col.names=F, row.names=FALSE)

# prom_file = "/nfs/lab/gencode_v19/promoters_by_gene.bed"
# outname = paste0('cicero/', cluster, '.intersect_promoter')
# command = paste( 'bedtools intersect -a bed_tempor -b', prom_file, '-wo >', outname)
# system(command)
                       
# int      = read.table(outname)
# ded      = int[ !duplicated(int[,1:7]),]
# ded$pair = str_split_fixed(ded$V4,"\\.",2)[,1]
# pp        =   unique(ded$pair [!( ded$pair %in%  ded[duplicated(ded[,c(5:7,11)]), "pair"])])
# long$prom = long$conn %in% int$V4
# ann       = cbind(long[1:nrow(ord), ], long[(nrow(ord)+1):nrow(long), ], dist)
# ann$same_promoter = !(1:nrow(ann) %in% as.numeric(pp)) & rowSums(ann[,c(5,10)])>1
# write.table (ann, paste0('cicero/', cluster, '.annotated_coaccess.txt') , sep='\t', quote=FALSE, row.names=FALSE)                      
                       
                       }

In [309]:
sumatab

NK-cell,12246031,306549
Monocyte,13425271,258360
CD8-T-cell,10777688,355772
CD4-T-cell,13199978,250390
B-cell,11550149,307134
pDC,5321601,872063
Megakaryocyte,4356379,231267


In [247]:
ann = read.table(paste0('cicero/', cluster, '.annotated_coaccess.txt'), header=T)

In [263]:
samp = str_split_fixed (rownames(sc.umap), "\\_",2)[,1]

In [265]:
uma = sc.umap[samp!='pbmc1',]

In [271]:
data.frame(table(uma$cluster))

Var1,Freq
<fct>,<int>
B-cell,3093
CD4-T-cell,9498
CD8-T-cell,2752
Megakaryocyte,445
Monocyte,10342
NK-cell,3968
pDC,195


In [260]:
pdf("cicero/summary_annot.pdf")
par(mfrow=c(2,3))
for(cluster in clusters){
ann = read.table(paste0('cicero/', cluster, '.annotated_coaccess.txt'), header=T)
tt= table(rowSums(ann[ann$span>1000,c(5,10,13)]))


barplot(tt, las=2, main=cluster, names.arg = c("CC", "CP", "PP", "PP_same"), col = cm.colors(4))
    }
dev.off()

png 
  2

In [311]:
annotations = data.frame()
for(cluster in clusters){
ann = read.table(paste0('cicero/', cluster, '.annotated_coaccess.txt'), header=T)
tt= table(rowSums(ann[ann$span>1000,c(5,10,13)]))
annotations = rbind(annotations, tt)

   }
colnames(annotations) = c("CC", "CP", "PP", "PP_same")
rownames(annotations) = clusters

In [314]:
annotations

,CC,CP,PP,PP_same
,<int>,<int>,<int>,<int>
NK-cell,107161,131950,62672,2365
Monocyte,111200,97311,44207,2531
CD8-T-cell,120748,155875,74688,2426
CD4-T-cell,93694,103885,47256,2637
B-cell,99366,135899,67165,2444
pDC,357302,380023,131755,1945
Megakaryocyte,57986,109405,61782,1313


### Check Jackie outputs

In [278]:
# setwd('/nfs/lab/projects/pbmc_snATAC/pipeline/snATAC/combined_files/test_jackie_cicero')
# wd = '/home/jnewsome/pipeline_islet/ciceroOutput'
# clusters = c('islet.Alpha_cell', 'islet.Beta_cell', 'islet.Delta_cell', 'islet.Ductal_cell', 'islet.Stellate_Cell')

In [315]:
wd = '/nfs/lab/projects/pbmc_snATAC/pipeline/snATAC/sample/PBMC1/lab_pipeline/cicero'
setwd(wd)

In [316]:
sc.umap <- read.table( 'pbmc1.cluster_labels.txt', sep='\t', header=T, row.names=1) ### map barcode to cluster 

In [298]:
data.frame(table(sc.umap$cluster))

Var1,Freq
<fct>,<int>
B-cell,1188
Megakaryocyte,210
Monocyte,3811
NK-cell,754
pDC,65
T-cell,4015


In [317]:
clusters = unique(sc.umap$cluster)

In [301]:
sumatab = matrix(0, ncol=2, nrow= length(clusters))
rownames(sumatab) = clusters
for (cluster in clusters){
dep = read.table(file.path(wd, paste0( 'pbmc1.',  cluster, '.cicero_conns_dedup.txt')), header=T, sep="\t")
dep = dep[complete.cases(dep),]
coa = dep[dep$coaccess >0.05, ]
sumatab[cluster,1] = nrow(dep)
sumatab[cluster,2] = nrow(coa)

    
bed = cbind(str_split_fixed(coa[,1], "\\_", 3 ), str_split_fixed(coa[,2], "\\_", 3 ))

### reorder the peaks (before they were character)
ord = matrix(parRapply(clus, bed, function(x) x[order(as.numeric(x))] ), ncol=6, byrow=T)
ord = ord[, c(5,1:2,5,3:4)]

dist = data.frame(span = as.numeric(ord[,6]) - as.numeric(ord[,2]), score = coa$coaccess)
#dist$size_bin = cut(dist$span, breaks = 5 )
#boxplot(score~size_bin, dist, las=2)
#nam = parRapply(clus, ord,  function(x) paste(x, collapse="_"))

long = data.frame(rbind(ord[,1:3], ord[,4:6]))

long$conn = paste( rep(1:nrow(ord),2),  rep(c('a','b'), each = nrow(ord)), sep=".")
write.table(long, "bed_tempor", sep="\t", quote=FALSE, col.names=F, row.names=FALSE)

prom_file = "/nfs/lab/gencode_v19/promoters_by_gene.bed"
outname = paste0( cluster, '.intersect_promoter')
command = paste( 'bedtools intersect -a bed_tempor -b', prom_file, '-wo >', outname)
system(command)
                       
int      = read.table(outname)
ded      = int[ !duplicated(int[,1:7]),]
ded$pair = str_split_fixed(ded$V4,"\\.",2)[,1]
pp        =   unique(ded$pair [!( ded$pair %in%  ded[duplicated(ded[,c(5:7,11)]), "pair"])])
long$prom = long$conn %in% int$V4
ann       = cbind(long[1:nrow(ord), ], long[(nrow(ord)+1):nrow(long), ], dist)
ann$same_promoter = !(1:nrow(ann) %in% as.numeric(pp)) & rowSums(ann[,c(5,10)])>1
write.table (ann, paste0( cluster, '.annotated_coaccess.txt') , sep='\t', quote=FALSE, row.names=FALSE)                      
                       
                       }

In [302]:
sumatab

NK-cell,1093164,55646
Monocyte,4095275,125667
T-cell,1853045,94694
B-cell,1622053,65332
pDC,296597,24874
Megakaryocyte,352094,29005


In [303]:
sumatab[,2]/sumatab[,1]

NK-cell      Monocyte        T-cell        B-cell           pDC 
   0.05090362    0.03068585    0.05110184    0.04027735    0.08386464 
Megakaryocyte 
   0.08237857

In [304]:
pdf("summary_annot.pdf")
par(mfrow=c(2,3))
for(cluster in clusters){
ann = read.table(paste0( cluster, '.annotated_coaccess.txt'), header=T)
#tt= table(rowSums(ann[ann$span>1000,c(5,10,13)]))
tt= table(rowSums(ann[,c(5,10,13)]))


barplot(tt, las=2, main=cluster, names.arg = c("CC", "CP", "PP", "PP_same"), col = cm.colors(4))
    }
dev.off()

png 
  2

In [320]:
annotations = data.frame()
for(cluster in clusters){
ann = read.table(paste0( cluster, '.annotated_coaccess.txt'), header=T)
tt= table(rowSums(ann[ann$span>1000,c(5,10,13)]))
annotations = rbind(annotations, tt)

   }
colnames(annotations) = c("CC", "CP", "PP", "PP_same")
rownames(annotations) = clusters

In [321]:
annotations

,CC,CP,PP,PP_same
,<int>,<int>,<int>,<int>
NK-cell,7827,19627,27426,692
Monocyte,36674,41947,44580,1685
T-cell,30947,34668,27510,1272
B-cell,14053,27895,22353,790
pDC,6591,11319,6847,90
Megakaryocyte,3610,11471,13728,160
